# Group E Assignment 5: NILMTK
## Introduction
Non-Intrusive Load Monitoring is an approach for the energy disaggregation which aims to help to predict individual appliances power and their behavior through the whole household power meter. One can directly conclude that there is no need of the individual sensors, which makes NILM a cheaper alternative to monitoring appliances. Secondly, this helps to reduce energy consumption, save money, find and flatten peak power loads and, finally, go to sustainability. 

In this project we are going to explore NILMTK Python-framework through the UKDALE dataset to perform energy disaggregation. 

## Exercise 1
In this exercise we mainly followed NILMTK documentation.

### Load UKDALE data into memory and print out the metadata

In [ ]:
from nilmtk import DataSet
from nilmtk.utils import print_dict
from nilmtk.timeframe import TimeFrame
import pandas as pd

ukdale = DataSet('./data/ukdale.h5')


#train = DataSet('/path/redd.h5')

### 1.2 Print out Metadata
Here we can see description of the dataset, e.g. which meter devices were used, location of the data reading, date, number of buildings, timeframe, creators etc. 

In [ ]:
print_dict(ukdale.metadata)


### Print out Buildings

In [ ]:
print_dict(ukdale.buildings)

### 1.3 Print out the sub-metered appliances in each building

Here we can see all submeters attached to appliances for each building separately. We conclude that each building has exactly one main meter  there are no nested MeterGroups for appliances.
Each submeter contains information about its instance, building, as well as appliance (type and instance)


In [ ]:
for build in ukdale.buildings:
    print("Appliances of Building " +str(build))
    print(ukdale.buildings[build].elec.submeters())
    print("---")
elec = ukdale.buildings[1].elec

### 1.4 Calculate the total energy consumption for building 1 in kWh

In [ ]:
elec.mains().total_energy()

### 1.5 Print out the type of power for mains and sub-meters
Same type of power is used for mains and submeters:

In [ ]:
elec.mains().available_ac_types('power')

In [ ]:
elec.submeters().available_ac_types('power')

## Exercise 2

### 2.1 Timeframed "Fridge Freezer" and "Light" Power Plot

In [ ]:
ukdale_window = DataSet('./data/ukdale.h5')
ukdale_window.set_window(start='2014-04-28', end='2014-04-29')

fridge_meter = ukdale_window.buildings[1].elec['fridge freezer']
light_meter = ukdale_window.buildings[1].elec['light']
elec = ukdale_window.buildings[1].elec

In [ ]:
fridge_meter.plot()
light_meter.plot()

### 2.2 Plot Overall Consumption For that time period

In [ ]:
all_window = next(ukdale_window.buildings[1].elec.load())
#all_window.head()

In [ ]:
all_window['power'].plot()

### 2.3 Calculate and plot the energy consumption fraction for each sub-meter


To get energy fraction per each submeter, we call `energy_per_meter()` method for the submeters. We are not interested in the reactive power since submeters do not utilize it, hence, we remove it from the consideration. Then we extract active and apparent power from the `energy_fraction_per_submeter` dataframe and build plots correspondingly.

In [ ]:
energy_fraction_per_submeter = elec.submeters().energy_per_meter().transpose().fillna(0)
del energy_fraction_per_submeter['reactive']
energy_fraction_per_submeter.div(energy_fraction_per_submeter.sum(axis=0))

#plot active power
energy_fraction_per_submeter['active'].plot(kind="bar", figsize=(15,10), title="Active Power Fraction")

In [ ]:
energy_fraction_per_submeter.plot(kind="bar", figsize=(15,10), title="Apparent Power Fraction")

### 2.4 Highest Power Consuming Appliance
Fridge freezer in the building 1 is the most power-consuming appliance. 

In [ ]:
max_appliance = elec.submeters().select_top_k(k=1).energy_per_meter()

### 2.5 Find appliances of the type “single-phase induction motor”

In [ ]:
elec.select_using_appliances(category='single-phase induction motor')

## Exercise 3
First steps:
 * set train set range until the end of the 24-03-2013
 * set test set range from the 25-03-2013

In [ ]:
import time
from six import iteritems
import matplotlib.pyplot as plt
import numpy as np

train = DataSet('./data/ukdale.h5')
test = DataSet('./data/ukdale.h5')

In [ ]:
train.set_window(end="24-3-2013")
test.set_window(start="25-3-2013")

train_elec = train.buildings[3].elec
test_elec = test.buildings[3].elec

#### Then goes data exploration: 
* First plot summarises power data for the building 3 for the site meter and various appliances during the training period
* Second plot indicates Site meter reading during the test period

In [ ]:
train_elec.plot()

In [ ]:
test_elec.mains().plot()

In [ ]:
mains = train_elec.mains()
mains_df = next(mains.load())
mains_df.head()

### Prepare a Method for Predicting and Calculating the F-Score


In [ ]:
from nilmtk.disaggregate import CombinatorialOptimisation, FHMM
from nilmtk.tests.testingtools import data_dir
from sklearn.metrics import f1_score

### Disaggregate and Calculate the F-Score With this function

Until the calculating F-score we mainly used used a sample code from the NILMTK documentation as for the reference for the model training and evaluation. Below the relevant steps are summarised:
*  Train the model on the training set
*  Read test data by chunks, apply the model to calculate predicted appliance power in the `pred` and ground truth data in the `gt`
*  Try to fit the data into the main memory by concatenating chunks into a pandas dataframe
*  Correspond data with the local timezone and put human-readable labels

As we are using F1 score the model evaluation, we transfer from the energy disaggregation to the classification problem. Hence, if the predicted power is more than a threshold, the device is classified as ON. 

F1 score is defined as a harmonic mean of the precision and recall metrics in the binary classification problem. 
Later we will explain the choice of the threshold value. 


In [ ]:
def disaggr_and_fscore(algorithm,train_elec, 
                       test_elec,train_timezone, 
                       threshold_w = 5,
                       show_debug=True):
    
    start = time.time()
    algorithm.train(train_elec,sample_period=6)
    end = time.time() 
    print("Train runtime =", end-start, "seconds.")
    pred = {}
    gt= {}

    for i, chunk in enumerate(test_elec.mains().load(sample_period=6)):
        chunk_drop_na = (chunk).dropna()
        pred[i] = algorithm.disaggregate_chunk(chunk_drop_na)
        gt[i]={}

        for meter in test_elec.submeters().meters:
            # Only use the meters that we trained on (this saves time!)    
            gt[i][meter] = next(meter.load(sample_period=6))
        gt[i] = pd.DataFrame({k:v.squeeze() for k,v in iteritems(gt[i])}, index=next(iter(gt[i].values())).index).dropna()

    gt_overall = pd.concat(gt)
    gt_overall.index = gt_overall.index.droplevel()
    pred_overall = pd.concat(pred)
    pred_overall.index = pred_overall.index.droplevel()

    gt_overall = gt_overall[pred_overall.columns]

    gt_index_utc = gt_overall.index.tz_convert("UTC")
    pred_index_utc = pred_overall.index.tz_convert("UTC")
    common_index_utc = gt_index_utc.intersection(pred_index_utc)


    common_index_local = common_index_utc.tz_convert(train_timezone)

    gt_overall = gt_overall.ix[common_index_local]
    pred_overall = pred_overall.ix[common_index_local]
    
    if show_debug:
        gt_overall.head()

    appliance_labels = [m.label() for m in gt_overall.columns.values]
    gt_overall.columns = appliance_labels
    pred_overall.columns = appliance_labels
    
    if show_debug:
        pred_overall.head()
        pred_overall.head(100000).plot(title="Pred",figsize=(15,5))
        gt_overall.head(100000).plot(title="GT",figsize=(15,5))
        plt.legend()

    resulting_f_score = {}
    #threshold_w = 5 #moved to function declaration
    for appliance in gt_overall.columns:
        temp_gt = gt_overall[appliance].copy()
        temp_gt[temp_gt<=threshold_w] = 0
        temp_gt[temp_gt>threshold_w] = 1
        temp_pred = pred_overall[appliance].copy()
        temp_pred[temp_pred<=threshold_w] = 0
        temp_pred[temp_pred>threshold_w] = 1
        resulting_f_score[appliance] = f1_score(temp_gt, temp_pred)

    return resulting_f_score

In [ ]:
classifiers = {'CO':CombinatorialOptimisation(), 'FHMM':FHMM()}
resulting_f_scores = {}

We are comparing two algorithms - Combinatorial Optimisation and Factorial Hidden Markov Model.


### 3.1 Combinatorial Optimisation

We now train the Combinatorial Optimisation model and apply it on the test data.  Below are two plots showing predicted and the ground truth power consumption.


In [ ]:
resulting_f_scores['CO'] = disaggr_and_fscore(CombinatorialOptimisation(),
                                              train_elec, 
                                              test_elec,train.metadata['timezone'],
                                              show_debug=True)

### 3.2 FHMM

The same for the FHMM model:

In [ ]:
resulting_f_scores['FHMM'] = disaggr_and_fscore(FHMM(),
                                                train_elec, 
                                                test_elec,
                                                train.metadata['timezone'],
                                                show_debug=False)

## Exercise 4

## Compare F-Score of CO and FHMM


In [ ]:
f_score_df={}
f_score_df['FHMM']=pd.Series(resulting_f_scores['FHMM'])
f_score_df['CO'] = pd.Series(resulting_f_scores['CO'])
f_score_df = pd.DataFrame(f_score_df)
f_score_df

In [ ]:
f_score_df.plot(kind='bar')

Here one can see F-scores for the individual appliance level using CO and FHMM algorithms.
CO performed significantly better for the kettle classification, showing F-score of `0.016418` against `0.009945`. It also showed slightly higher F-metric for the Electric space heater and the Laptop computer but not that significant. In contrast, FHMM has an F-score of 0.091067 for the projector versus 0.087560 by CO but the diffence is very small. Thus, we conclude that Combinatorial Optimisation performed better. 

Now why we chose a threshold of 5. It was actually retrieved empirically since a valud of 0 yielded inadequate results:

We chose empirically a classification threshold to be 5 since a threshold of 0 gave inadequate F-score:



| Appliance        | CO           | FHMM  |
| ------------- |-------------| -----|
| Electric space heater|	0.189193	|1.000000 |
| Kettle|	0.396679|	0.912690 |
| Laptop computer|	0.397019|	0.375311|
| Projector|	0.088132|	0.091067 |

## Conclusion and findings

foo